# Preprocessing of data

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
## data ingestion
#import libraries and creating connection with Mongodb
import pymongo
import json
client = pymongo.MongoClient("localhost")
print(client)
#defining database and collection
db = client["Regression"]
collection = db["household_power_consumption"]
#Fetching data from database
data = list(collection.find({},{'_id':0}))
df = pd.DataFrame(data)
df

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,month,total_metering
0,0.210,0.084,228.56,1.0,7,0.0
1,3.122,0.132,242.14,12.8,12,19.0
2,1.830,0.458,242.12,7.8,5,19.0
3,1.754,0.188,239.06,7.4,5,18.0
4,1.272,0.250,241.00,5.4,7,1.0
...,...,...,...,...,...,...
14995,0.000,0.000,0.00,0.0,8,0.0
14996,2.266,0.066,241.81,9.4,1,18.0
14997,0.524,0.062,238.84,2.2,9,0.0
14998,1.378,0.150,237.61,5.8,9,19.0


## Splitting of dependent and independent data

In [3]:
## Independent features
X = df.drop(["total_metering"],axis=1)
X

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,month
0,0.210,0.084,228.56,1.0,7
1,3.122,0.132,242.14,12.8,12
2,1.830,0.458,242.12,7.8,5
3,1.754,0.188,239.06,7.4,5
4,1.272,0.250,241.00,5.4,7
...,...,...,...,...,...
14995,0.000,0.000,0.00,0.0,8
14996,2.266,0.066,241.81,9.4,1
14997,0.524,0.062,238.84,2.2,9
14998,1.378,0.150,237.61,5.8,9


In [4]:
## Dependent feature
y = df["total_metering"]
y

0         0.0
1        19.0
2        19.0
3        18.0
4         1.0
         ... 
14995     0.0
14996    18.0
14997     0.0
14998    19.0
14999     0.0
Name: total_metering, Length: 15000, dtype: float64

## Splitting of training and testing data

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=48,test_size=0.33)

In [6]:
x_train.shape

(10050, 5)

In [7]:
y_train.shape

(10050,)

In [8]:
x_test.shape

(4950, 5)

In [9]:
y_test.shape

(4950,)

## Standization of DATA

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_train

array([[-0.84369277,  0.25634068,  0.09358889, -0.8444995 ,  0.46133945],
       [ 0.02105447,  0.32773141,  0.05625804,  0.00681677,  0.16868898],
       [-0.86818452, -1.08223554,  0.10920689, -0.88930562,  0.46133945],
       ...,
       [-0.79282529, -1.08223554,  0.10958782, -0.79969338,  0.75398993],
       [-0.76833353, -0.15415603,  0.22081852, -0.75488726,  0.16868898],
       [-0.85688064, -1.08223554, -0.0907798 , -0.8444995 , -0.1239615 ]])

In [11]:
#mean calculated for every feature
scaler.mean_

array([1.07764896e+00, 1.21274627e-01, 2.37973128e+02, 4.56957214e+00,
       6.42358209e+00])

## Pickeling of Objects

In [12]:
import pickle as pk
scaler_pk = pk.dumps(scaler)
x_train_pk = pk.dumps(x_train)
y_train_pk = pk.dumps(y_train)
x_test_pk = pk.dumps(x_test)
y_test_pk = pk.dumps(y_test)

## DB Operations

## Restoring data in to mongo db

In [15]:
## Deleting old data
db = client["Regression"]
if "household_power_consumption_pickles" in db.list_collection_names():
    db.drop_collection("household_power_consumption_pickles")
#defining collection
collection = db["household_power_consumption_pickles"]
#Storing data into database
collection.insert_one({"scaler_obj":scaler_pk})
collection.insert_one({"Train_data":[x_train_pk,y_train_pk]})
collection.insert_one({"Test_data":[x_test_pk,y_test_pk]})